<a href="https://colab.research.google.com/github/baveet256/NLP-hands-on/blob/main/Fake_news_classifier(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# data unzipping

In [59]:
!unzip /content/drive/MyDrive/Fake_News_Dataset/fake-news.zip -d /content/drive/MyDrive/Unzipped_fake_news

Archive:  /content/drive/MyDrive/Fake_News_Dataset/fake-news.zip
replace /content/drive/MyDrive/Unzipped_fake_news/submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/Unzipped_fake_news/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/Unzipped_fake_news/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [60]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Unzipped_fake_news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [61]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#  becuz of no info , eliminating the null values

In [62]:
df = df.dropna()

In [63]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [64]:
X,y = df.drop('label',axis=1),df['label']

In [65]:
X['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [67]:
# pre-processing the data

#preprocessing

In [68]:
# doing word2vec here, as it previous knowledge felt alienated now.,
# doing stopword process and stemming/lemmatization
import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
messages=X.copy()
data_size = len(messages)
messages.reset_index(inplace=True)

In [70]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [71]:
for i in range(5):
  print(i) 

0
1
2
3
4


In [72]:

from nltk.stem.porter import PorterStemmer ##stemming purpose
port_stem = PorterStemmer()
corpus = []
#parsing the data samples.
for i in range(0,data_size):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()  
  review = [port_stem.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [73]:
corpus[5]

'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart'

In [89]:
# 
voc_size = 10000
# words gives the rank or index of the word in the dictionary size of 10k
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[9672, 5080, 5423, 9259, 3758, 2056, 8263, 3476, 5660, 5251],
 [5397, 1919, 2591, 6839, 1843, 9673, 1183],
 [8033, 1860, 8383, 1061],
 [7859, 4319, 3460, 8168, 9120, 9652],
 [8465, 1843, 1403, 5773, 8655, 8258, 1843, 6042, 9440, 9848],
 [9669,
  4732,
  119,
  7650,
  4568,
  4693,
  5976,
  2418,
  5058,
  8919,
  9447,
  8339,
  2439,
  2031,
  1183],
 [6801, 862, 1605, 423, 3191, 5631, 6210, 4382, 8116, 3204, 7972],
 [3141, 9468, 1644, 6318, 5258, 4350, 4693, 7565, 8116, 3204, 7972],
 [5469, 9730, 3064, 5002, 3661, 4656, 8301, 8846, 4693, 9467],
 [7449, 4222, 1591, 5443, 693, 8410, 9774, 2987],
 [1988, 2091, 9763, 669, 1479, 9473, 7317, 4977, 265, 6827, 4995],
 [8168, 2585, 3758, 4656, 4693, 5258],
 [4457, 3949, 9754, 9766, 2840, 2504, 32, 3162, 4094],
 [7641, 7202, 9316, 5280, 9143, 9831, 4834, 8116, 3204, 7972],
 [1468, 5766, 1734, 2952, 2193, 8116, 3204, 7972],
 [2209, 585, 2818, 6833, 1518, 9324, 3743, 5637, 7593, 4998],
 [345, 8937, 1919],
 [7552, 5762, 6950, 1223, 4693, 750, 

In [77]:
print(f'{corpus[5]} \n')
print(f'{onehot_repr[5]} \n')

jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart 

[9669, 4732, 119, 7650, 4568, 4693, 5976, 2418, 5058, 8919, 9447, 8339, 2439, 2031, 1183] 



We do padding , to make every sentences , an equal length

In [82]:

maxi=0

index = 0
for i in range(0,data_size):
  curr = i
  curr_len =  len(onehot_repr[curr])
  if curr_len > maxi :
    maxi = len(onehot_repr[curr])
    index = i

print(maxi)
print(index)  

47
1020


In [86]:
median_len = 20
count = 0
for i in range(0,data_size):
  curr = i
  curr_len =  len(onehot_repr[curr])
  if curr_len > median_len:
    count+=1
count

37

In [ ]:
# so optimum padding should be less than 20, becuz max one_hot is of length 47 and just 37 are  (> 20,< 47)

In [90]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3476 5660 5251]
 [   0    0    0 ... 1843 9673 1183]
 [   0    0    0 ... 1860 8383 1061]
 ...
 [   0    0    0 ... 8116 3204 7972]
 [   0    0    0 ... 9612 3551 6075]
 [   0    0    0 ... 3734 5914 8368]]


In [92]:
print(embedded_docs[5])
print(onehot_repr[5])
print(corpus[5])

[   0    0    0    0    0 9669 4732  119 7650 4568 4693 5976 2418 5058
 8919 9447 8339 2439 2031 1183]
[9669, 4732, 119, 7650, 4568, 4693, 5976, 2418, 5058, 8919, 9447, 8339, 2439, 2031, 1183]
jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart


In [94]:
##features representation
embedding_vector_features=32 
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(250))
model.add(Dense(1,activation='sigmoid'))

In [100]:
#using cross rntropy and ADAM optimizer
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [101]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 32)            320000    
                                                                 
 lstm (LSTM)                 (None, 250)               283000    
                                                                 
 dense (Dense)               (None, 1)                 251       
                                                                 
Total params: 603,251
Trainable params: 603,251
Non-trainable params: 0
_________________________________________________________________
None


In [102]:
len(embedded_docs),y.shape

(18285, (18285,))

In [103]:
import numpy as np
X_now=np.array(embedded_docs)
y_now=np.array(y)

In [104]:
# 90-10 split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_now, y_now, test_size=0.1, random_state=42)

In [105]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=32)

Epoch 1/12
515/515 [==============================] - 30s 53ms/step - loss: 0.2458 - accuracy: 0.8882 - val_loss: 0.2164 - val_accuracy: 0.9120
Epoch 2/12
515/515 [==============================] - 26s 50ms/step - loss: 0.1155 - accuracy: 0.9555 - val_loss: 0.2152 - val_accuracy: 0.9196
Epoch 3/12
515/515 [==============================] - 27s 53ms/step - loss: 0.0667 - accuracy: 0.9757 - val_loss: 0.2571 - val_accuracy: 0.9191
Epoch 4/12
515/515 [==============================] - 25s 48ms/step - loss: 0.0384 - accuracy: 0.9875 - val_loss: 0.3126 - val_accuracy: 0.9163
Epoch 5/12
515/515 [==============================] - 26s 51ms/step - loss: 0.0198 - accuracy: 0.9936 - val_loss: 0.3681 - val_accuracy: 0.9158
Epoch 6/12
515/515 [==============================] - 26s 50ms/step - loss: 0.0151 - accuracy: 0.9953 - val_loss: 0.4201 - val_accuracy: 0.9131
Epoch 7/12
515/515 [==============================] - 25s 49ms/step - loss: 0.0089 - accuracy: 0.9971 - val_loss: 0.5252 - val_accuracy:

## So, we have val accuracy of 91.47

Now, using the CNN Model